# Chapter 26: Bit Manipulation

> *"Bits are the atoms of computation. Mastering bit manipulation gives you the power to perform complex operations with elegance and speed."* — Anonymous

---

## 26.1 Introduction to Bit Manipulation

**Bit manipulation** refers to the art of algorithmically altering individual bits (the smallest unit of data) using bitwise operators. It is a fundamental skill in low-level programming, embedded systems, cryptography, graphics, and performance-critical applications.

### 26.1.1 Why Bit Manipulation Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF BIT MANIPULATION                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. PERFORMANCE: Bit operations are extremely fast (single CPU      │
│     instruction) compared to arithmetic or branching.               │
│                                                                      │
│  2. SPACE EFFICIENCY: Represent multiple boolean flags in one       │
│     integer, saving memory.                                         │
│                                                                      │
│  3. EMBEDDED SYSTEMS: Direct hardware control requires setting      │
│     bits in registers.                                              │
│                                                                      │
│  4. CRYPTOGRAPHY: Many encryption algorithms rely on bitwise        │
│     XOR, shifting, and masking.                                     │
│                                                                      │
│  5. GRAPHICS: Color channels, bitmaps, and pixel operations.        │
│                                                                      │
│  6. COMPRESSION: Huffman coding, arithmetic coding use bit-level    │
│     operations.                                                      │
│                                                                      │
│  7. ALGORITHMIC TRICKS: Many elegant solutions use bit manipulation │
│     (e.g., finding missing numbers, subset enumeration).            │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 26.2 Bitwise Operators

Most programming languages provide the following bitwise operators (here using Python syntax):

| Operator | Name            | Description                                      |
|----------|-----------------|--------------------------------------------------|
| `&`      | AND             | Sets bit to 1 if both bits are 1                 |
| `\|`     | OR              | Sets bit to 1 if at least one bit is 1           |
| `^`      | XOR             | Sets bit to 1 if exactly one bit is 1 (exclusive OR) |
| `~`      | NOT             | Flips all bits (one's complement)                 |
| `<<`     | Left shift      | Shifts bits left, fills with 0                   |
| `>>`     | Right shift     | Shifts bits right (sign extension in some languages, in Python it's logical shift) |

**Note:** In Python, integers are unbounded and `~x` yields the two's complement with infinite bits, which can be surprising. For positive numbers, `~x` = `-x-1`.

### 26.2.1 Truth Tables

```
AND: 0&0=0, 0&1=0, 1&0=0, 1&1=1
OR:  0|0=0, 0|1=1, 1|0=1, 1|1=1
XOR: 0^0=0, 0^1=1, 1^0=1, 1^1=0
```

### 26.2.2 Shifts

- `x << k` multiplies x by `2^k`.
- `x >> k` (in Python) performs floor division by `2^k` (logical shift for non-negative, preserves sign for negative but behavior is implementation-defined in some languages). In Python, right shift on negative numbers is arithmetic shift (fills with 1's), but since Python integers are infinite, the result is still negative.

---

## 26.3 Basic Bit Tricks and Idioms

Let `x` be an integer (considering its binary representation).

### 26.3.1 Check if a number is even or odd

```python
if x & 1:
    print("odd")
else:
    print("even")
```

### 26.3.2 Set the k-th bit (from 0)

```python
x = x | (1 << k)
```

### 26.3.3 Clear (unset) the k-th bit

```python
x = x & ~(1 << k)
```

### 26.3.4 Toggle the k-th bit

```python
x = x ^ (1 << k)
```

### 26.3.5 Check if k-th bit is set

```python
if x & (1 << k):
    print("set")
```

### 26.3.6 Get the lowest set bit

```python
lowest = x & -x
```

This works because `-x` is two's complement of x; it flips all bits and adds 1, leaving only the lowest set bit.

### 26.3.7 Clear all bits except the lowest set

Already done: `x & -x`.

### 26.3.8 Check if a number is a power of two

```python
def is_power_of_two(x):
    return x > 0 and (x & (x - 1)) == 0
```

Because powers of two have exactly one bit set; `x-1` flips all lower bits, so AND becomes zero.

### 26.3.9 Count trailing zeros

```python
def count_trailing_zeros(x):
    # Using built-in (Python 3.11+ has int.bit_count, but not trailing zeros)
    # We can do: return (x & -x).bit_length() - 1
    return (x & -x).bit_length() - 1
```

Alternatively, use a loop.

### 26.3.10 Count leading zeros

In Python, `x.bit_length()` gives number of bits needed to represent x, so leading zeros = total bits (e.g., 32) minus `bit_length()` if we consider fixed width.

### 26.3.11 Swap two numbers without temporary variable

```python
a = a ^ b
b = a ^ b
a = a ^ b
```

This works because XOR is its own inverse.

### 26.3.12 Add two numbers using bitwise operations

```python
def add(a, b):
    while b:
        carry = a & b
        a = a ^ b
        b = carry << 1
    return a
```

---

## 26.4 Bit Masking and Subset Enumeration

Bit masking uses an integer to represent a set of items (usually indices). The k-th bit set means item k is present.

### 26.4.1 Representing a subset

For a set of n elements, any subset can be represented by an n-bit integer (bits 0..n-1).

### 26.4.2 Iterating over all subsets

```python
n = 4
for mask in range(1 << n):
    subset = [i for i in range(n) if mask & (1 << i)]
    print(mask, subset)
```

### 26.4.3 Iterating over all subsets of a given mask (submask enumeration)

To iterate over all submasks of a given mask `m`:

```python
sub = m
while sub:
    # process submask
    sub = (sub - 1) & m
# also consider sub=0 separately if needed
```

This trick enumerates all submasks in decreasing order.

### 26.4.4 Check if a mask is superset of another

```python
if (a & b) == b:  # b is subset of a
    ...
```

### 26.4.5 Set union and intersection

- Union: `a | b`
- Intersection: `a & b`
- Symmetric difference: `a ^ b`

---

## 26.5 XOR Properties and Applications

XOR has several useful properties:

- Identity: `a ^ 0 = a`
- Self-inverse: `a ^ a = 0`
- Commutative: `a ^ b = b ^ a`
- Associative: `(a ^ b) ^ c = a ^ (b ^ c)`

### 26.5.1 Find the missing number

Given an array of n-1 numbers containing all numbers from 1 to n except one, find the missing number.

```python
def missing_number(arr, n):
    xor_all = 0
    for i in range(1, n+1):
        xor_all ^= i
    for num in arr:
        xor_all ^= num
    return xor_all
```

### 26.5.2 Find the single non-repeating element

Given an array where every element appears twice except one, find that single element.

```python
def single_number(arr):
    result = 0
    for num in arr:
        result ^= num
    return result
```

### 26.5.3 Find two non-repeating elements

Given an array where every element appears twice except two, find those two.

```python
def two_single_numbers(arr):
    xor = 0
    for num in arr:
        xor ^= num
    # xor now equals a ^ b, where a and b are the two singles
    # Find a set bit (rightmost)
    diff_bit = xor & -xor
    a = b = 0
    for num in arr:
        if num & diff_bit:
            a ^= num
        else:
            b ^= num
    return a, b
```

### 26.5.4 Find the missing and duplicate number

Given an array of size n containing numbers from 1 to n, but one is missing and one is duplicated (each exactly once), find them.

```python
def missing_and_duplicate(arr, n):
    xor = 0
    for i in range(1, n+1):
        xor ^= i
    for num in arr:
        xor ^= num
    # xor = missing ^ duplicate
    diff_bit = xor & -xor
    a = b = 0
    for i in range(1, n+1):
        if i & diff_bit:
            a ^= i
        else:
            b ^= i
    for num in arr:
        if num & diff_bit:
            a ^= num
        else:
            b ^= num
    # Now a and b are missing and duplicate; we need to determine which is which.
    if a in arr:
        return b, a  # b missing, a duplicate
    else:
        return a, b
```

---

## 26.6 Brian Kernighan's Algorithm (Counting Set Bits)

This algorithm counts the number of 1-bits by repeatedly clearing the lowest set bit.

```python
def count_set_bits(n):
    count = 0
    while n:
        n = n & (n - 1)  # clear lowest set bit
        count += 1
    return count
```

**Time:** O(number of set bits).  
Python also has built-in `int.bit_count()` (Python 3.8+).

---

## 26.7 Advanced Bit Manipulation Techniques

### 26.7.1 Reverse bits

Reverse the bits of an integer (e.g., for fixed-width numbers).

```python
def reverse_bits(n, bit_width=32):
    result = 0
    for i in range(bit_width):
        result = (result << 1) | (n & 1)
        n >>= 1
    return result
```

A more efficient approach uses divide-and-conquer with masks (like in Hacker's Delight).

### 26.7.2 Swap odd and even bits

Swap all odd and even bits positions (e.g., bit0 swaps with bit1, bit2 with bit3, etc.):

```python
def swap_odd_even_bits(x):
    # Mask for odd bits (1010...), even bits (0101...)
    odd_mask = 0xAAAAAAAA  # for 32-bit, adjust for wider
    even_mask = 0x55555555
    return ((x & odd_mask) >> 1) | ((x & even_mask) << 1)
```

### 26.7.3 Check if a number has alternating bits

```python
def has_alternating_bits(n):
    # n ^ (n >> 1) should have all bits set if alternating
    x = n ^ (n >> 1)
    return (x & (x + 1)) == 0
```

### 26.7.4 Gray code

Gray code is a binary numeral system where two successive values differ in only one bit. `n ^ (n >> 1)` generates Gray code for n.

### 26.7.5 Next power of two

Find the smallest power of two greater than or equal to n.

```python
def next_power_of_two(n):
    if n <= 0:
        return 1
    n -= 1
    n |= n >> 1
    n |= n >> 2
    n |= n >> 4
    n |= n >> 8
    n |= n >> 16
    return n + 1
```

---

## 26.8 Applications in Real-World

- **Graphics:** Color manipulation (ARGB channels packed into 32-bit integers).
- **Permissions:** Unix file permissions (rwx as bits).
- **Compression:** Huffman codes are bit sequences.
- **Cryptography:** AES, DES heavily use XOR and shifts.
- **Networking:** IP addresses and subnet masks.
- **Embedded:** Setting configuration registers in microcontrollers.
- **Game development:** Representing game states (e.g., chess board as bitboards).

---

## 26.9 Summary

```
┌─────────────────────────────────────────────────────────────────────┐
│                    BIT MANIPULATION SUMMARY                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Operators: &, |, ^, ~, <<, >>                                      │
│                                                                      │
│  Basic Tricks:                                                      │
│    • Check even: (x & 1) == 0                                       │
│    • Set bit k: x | (1 << k)                                        │
│    • Clear bit k: x & ~(1 << k)                                     │
│    • Toggle bit k: x ^ (1 << k)                                     │
│    • Lowest set bit: x & -x                                         │
│    • Power of two: x > 0 and (x & (x-1)) == 0                       │
│    • Count bits: while n: n &= n-1; cnt++                           │
│                                                                      │
│  XOR Properties: a^a=0, a^0=a, commutative, associative             │
│                                                                      │
│  Subset Enumeration: iterate mask 0..2^n-1, submask enumeration     │
│                                                                      │
│  Common Problems: missing number, single number, two singles,       │
│  missing & duplicate, reverse bits, swap odd/even bits.             │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 26.10 Practice Problems

### Easy
1. **Single Number** (LeetCode 136)
2. **Number of 1 Bits** (LeetCode 191)
3. **Power of Two** (LeetCode 231)
4. **Missing Number** (LeetCode 268)
5. **Reverse Bits** (LeetCode 190)

### Medium
6. **Single Number II** (LeetCode 137) – each element appears 3 times except one.
7. **Single Number III** (LeetCode 260) – two elements appear once.
8. **Subsets** (LeetCode 78) – using bitmask enumeration.
9. **Maximum Product of Word Lengths** (LeetCode 318) – use bitmask for letters.
10. **Counting Bits** (LeetCode 338) – DP with bits.

### Hard
11. **Minimum XOR** (not on LC but classic) – find two numbers with minimum XOR.
12. **Maximum XOR of Two Numbers in an Array** (LeetCode 421) – use trie.
13. **Find the Duplicate Number** (LeetCode 287) – can be solved with bit manipulation.
14. **UTF-8 Validation** (LeetCode 393) – bit pattern matching.
15. **IP to CIDR** (LeetCode 751) – bit operations on IP addresses.

---

## 26.11 Further Reading

1. **"Hacker's Delight"** by Henry S. Warren – The definitive book on bit manipulation.
2. **"The Art of Computer Programming, Vol 4A"** by Donald Knuth – Section on bitwise tricks.
3. **"Bit Twiddling Hacks"** by Sean Eron Anderson – online collection of tricks.
4. **Original Papers/Articles:**
   - Kernighan, B. W., & Ritchie, D. M. – "The C Programming Language" (includes bitwise operations).
   - Warren, H. S. – "Hacker's Delight" (book).

---

> **Coming in Chapter 27**: **Computational Geometry** – We'll explore algorithms for geometric problems: convex hull, line intersection, closest pair, and more.

---

**End of Chapter 26**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../7. string_algorithms/25. advanced_string_structures.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='27. computational_geometry.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
